In [ ]:
%matplotlib inline

import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np

mlp.style.use('ggplot')

# Neural Network - 手書き文字認識

以下の手順で進みます

- データの確認
- Neural Networkの作成
  - 活性化関数
  - 情報の伝搬処理
  - 各ユニットの初期値

参考：[ニューラルネットワークで数字を認識するWebアプリを作る](http://qiita.com/ginrou@github/items/07b52a8520efcaebce37)

## データの確認

- 入力層のサイズ：784次元 (画像サイズ：28x28)
- 出力層のサイズ：10次元 (0-9)
- 中間層のサイズ：300次元

In [ ]:
mnist = np.load('data/mnist.npz')
x_train = mnist['x_train']
x_test = mnist['x_test']
y_train = mnist['y_train']
y_test = mnist['y_test']

In [ ]:
# 訓練データのサイズ
x_train.shape

In [ ]:
# テストデータのサイズ
x_test.shape

In [ ]:
# データの中身はどうなっているのか？
print(x_train[25000][:90])
print(y_train[25000])

In [ ]:
def draw_digit(data):
    size = 28
    X, Y = np.meshgrid(range(size), range(size))
    Z = data.reshape(size, size)
    Z = Z[::-1,:]

    plt.figure(figsize=(2.5, 2.5))
    
    plt.xlim(0, 27)
    plt.ylim(0, 27)
    plt.pcolor(X, Y, Z)
    plt.gray()
    plt.tick_params(labelbottom="off")
    plt.tick_params(labelleft="off")
    
    plt.show()

# 個々のデータを描画してみる

draw_digit(x_train[0])
draw_digit(x_train[40000])
draw_digit(x_train[-1])

## Neural Networkの作成

実際にNeural Networkを設計してみる

- 活性化関数
- 情報の伝搬処理
- 重みの更新 ( ※時間的な問題で今回説明しません )

### 活性化関数

In [ ]:
# シグモイド関数

f = lambda x: 1. / (1. + np.exp(-x))

In [ ]:
# 関数の形を描画してみる

x = [xi for xi in np.linspace(-10, 10)]
y = [f(xi) for xi in x]

plt.plot(x, y, 'o--')

### 情報の伝播処理

In [ ]:
def fire(x, hidden_weight, output_weight):
    z = np.vectorize(f)(hidden_weight.dot(np.r_[np.array([1]), x]))
    y = np.vectorize(f)(output_weight.dot(np.r_[np.array([1]), z]))
    return (z, y)

### 重みの更新 ( ※時間的な問題で今回説明しません )

In [ ]:
def fit(x, t, hidden_weight, output_weight, learning_ratio=0.1):
    z, y = fire(x, hidden_weight, output_weight)
    dy = (y - t) * y * (1 - y)
    dz = (output_weight.T.dot(dy))[1:] * z * (1 - z)

    hidden_input = np.r_[np.array([1]), x]
    hidden_weight -= learning_ratio * dz.reshape(-1, 1) * hidden_input
    
    output_input = np.r_[np.array([1]), z]
    output_weight -= learning_ratio * dy.reshape(-1, 1) * output_input
 
    return hidden_weight, output_weight

### 予測処理

In [ ]:
def predicate(x, hidden_weight, output_weight):
    z, y = fire(x, hidden_weight, output_weight)
    return np.array(y).argmax()

## 作成したNeural Networkで実験

作成したNeural Networkを使って文字認識の実験をしてみましょう

- 学習
- 予測

### 学習

In [ ]:
input_size = 784
output_size = 10
hidden_size = 300

# 重みを一様分布でおくと精度がめっちゃ下がるの面白かったので残しておく (50%位になる)
#hidden_weight = np.random.uniform(-1.0, 1.0, (hidden_size, input_size + 1))
#output_weight = np.random.uniform(-1.0, 1.0, (output_size, hidden_size + 1))

hidden_weight = 0.1 * (np.random.random_sample((hidden_size, input_size+1)) - 0.5)
output_weight = 0.1 * (np.random.random_sample((output_size, hidden_size+1)) - 0.5)

print(hidden_weight.shape)
print(output_weight.shape)

In [ ]:
def to_formatted_array(x, output_size):
    formatted_x = np.zeros(output_size)
    formatted_x.put(x, 1)
    return formatted_x

In [ ]:
for i in range(x_train.shape[0]):
    x = x_train[i]
    y = to_formatted_array(y_train[i], output_size)

    hidden_weight, output_weight = fit(x, y, hidden_weight, output_weight)

### 予測

In [ ]:
ok = 0

length = x_test.shape[0]
for i in range(length):
    x = x_test[i]
    y = y_test[i]

    pred = predicate(x, hidden_weight, output_weight)
    if int(y) == int(pred):
        ok += 1

print("{0:05d} / {1:05d} = {2:3.2f}%".format(ok, length, 100. * ok / length))

## 作ったモデルを保存

In [ ]:
np.savez('data/neural_network', hidden=hidden_weight, output=output_weight)